In [5]:
import numpy as np
import pandas as pd

# Exponential Mixture Model

En esta notebook se realizará un modelo similar al Guassian Mixture Model (de aquí en más GMM) pero utilizando en vez de una distribución gaussiana, una exponencial. 


In [33]:
class ExponentialMixtureModel():
    def __init__(self,k,max_iter=100, tol=1e-6):
        #Cantidad de Clusters
        self.k = k
        #Maxima cantidad de iteraciones que el algoritmo va a hacr
        self.max_iter = max_iter
        #Tolerancia que utilizará el algoritmo para comprar entre los log-likelihood
        self.tol = tol
        #Vector de Pi's/ pesos de cada feature
        self.weight = None
        self.parametro = None

    
    def initialize_parameters(self,X): #Inicialización de los parámetros de forma random
        n_samples= X.shape[0]
        #Se fija los pesos de forma uniforme
        self.weight=np.full(self.k, 1/ n_samples)
        self.parametro = np.array([np.random.uniform(0, 10),np.random.uniform(0, 10)])


    def e_step(self,X): #En este paso se calcula la probabilidad de pertenecer al cluster=j dado las X ponderandolas 
                        #por el peso de cada cluster, inicializado anteriormanete. 
                        # Es una regla de bayes ponderada con la exponencial como fx
        n_samples = X.shape[0]
        #Generamos la matriz de respuesta del e-step con todos 0.
        #La matriz es de n x k 
        self.e_return = np.zeros((n_samples, self.k))
        #Suma de 1 a k clusters de 
        for k in range(self.k):
            self.e_return[:, k] = self.weight[k] * self.parametro[k] * np.exp(-self.parametro[k] * X)
        #Normalización de las probabilidad para que la proababilidad de cada observación de pertenecer a cada k sume 1
        self.e_return = self.e_return / self.e_return.sum(axis=1, keepdims=True) 

    
    def m_step(self,X):
        n_samples= X.shape[0]
        #Actualización de los pesos dados los parámetros. Se hace una suma de 1 a k y de todas las filas
        for k in range(self.k):
            #Se obtiene la suma de wij que es la probabilidad codicional normalizada obtenida en el e_step y se la dividie por n
            #Se tiene que obtener tantos 

            self.weight[k]= np.sum(self.e_return[:,k]) / n_samples
            #Una vez que tenemos los pesos actualizamos el parámetro lambda
            self.parametro[k] = np.sum(self.e_return[:,k])  / np.sum(self.e_return[:, k] * X)

    
    def fit(self,X):
         self.initialize_parameters(X)
         log_likelhood_old= 0.0
         for i in range(self.max_iter):
            self.e_step(X)  # Calcular las probabilidades de que cada observación pertenezca a k=j clase--> Paso E
            self.m_step(X)  # Actualizar los parámetros --> Paso M
            # Calcular la log-verosimilitud
            log_likelihood = np.sum(np.log(np.sum(self.e_return, axis=1)))
            if np.abs(log_likelihood - log_likelhood_old) < self.tol:
                break
            log_likelhood_old = log_likelihood


    
    def predict_proba(self,X): #Calcula las probabilidad de pertenecer a la ultima clase dado la maximización
        self.e_step(X)
        
        return self.e_return
    
    def predict(self,X): #devuelve la clase con mayor probabilidad para cada observación


        return np.argmax(self.predict_proba(X), axis=1)
        



        

        



        


# Ejemplo de uso

In [34]:
np.random.seed(0)
X = np.concatenate([
    np.random.exponential(scale=1/2, size=100),  
    np.random.exponential(scale=1/5, size=100) 
])

# Crear y ajustar el modelo
emm = ExponentialMixtureModel(k=2,max_iter=1000)
emm.fit(X)

# Predicciones
print("Pesos de mezcla:", emm.weight )
print("Tasas (lambda):", emm.parametro)
clusters = emm.predict(X)
print("Asignación de clusters:", clusters)
print("Matriz de probabilidad:",emm.predict_proba(X))

Pesos de mezcla: [0.55961478 0.44038522]
Tasas (lambda): [2.18096379 5.63049628]
Asignación de clusters: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 1
 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 0 1 0
 0 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0
 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1]
Matriz de probabilidad: [[6.60128203e-01 3.39871797e-01]
 [8.11127808e-01 1.88872192e-01]
 [7.07549509e-01 2.92450491e-01]
 [6.56763906e-01 3.43236094e-01]
 [5.60107397e-01 4.39892603e-01]
 [7.46825051e-01 2.53174949e-01]
 [5.70478622e-01 4.29521378e-01]
 [9.57961751e-01 4.20382492e-02]
 [9.93364191e-01 6.63580918e-03]
 [5.31276280e-01 4.68723720e-01]
 [8.80497227e-01 1.19502773e-01]
 [6.43217138e-01 3.56782862e-01]
 [6.76775731e-01 3.23224269e-01]
 [9.77523437e-01 2.24765627e-02]
 [